# FreeCAD Jupyter translation

First we load required libraries for the tranlation from FreeCAD Coin3D scene graph to a pythreejs WebGL rendering. And even before that make sure you have a working FreeCAD and python3 setup. Then install pythreejs:

```
pip3 install pythreejs
jupyter nbextension install --py --symlink --sys-prefix pythreejs
jupyter nbextension enable --py --sys-prefix pythreejs   
```

In [1]:
from pivy import coin
from pythreejs import *
import numpy as np
from ipywidgets import HTML

The following functions will do the job, in the end `render_object` will be all that you need to know in order to use this feature.

For this we need to iterate over the scene graph and extract the edges and surfaces. In this setting by default we only grab the face representation (by chosing one of the switch node children. The switch node allows switching between different FreeCAD views such as Mesh, Surface etc.)

To view the scene graph structure in a convenient way open `FreeCAD > Tools > view scene graph` in the FUI. That's how I verified the locations of `SoIndexedFaceSet` and the corresponding `SoCoordinate3` object. The coordinates all switch children refer to are always the coordinates in the root of the object. So one level below the document root node.

In [2]:
HIGHLIGHTING_COLOR = (0,1,0)
LINE_WIDTH = 10

def so_col_to_hex(so_color):
    """
    Translate Coin scene object color into html hex color strings
    """
    color = (int(so_color[0]*255), 
                  int(so_color[1]*255),
                  int(so_color[2]*255))
    hex_col = "#{0:02x}{1:02x}{2:02x}".format(color[0],
                                              color[1],
                                              color[2])
    return hex_col

def transform_indices(so_node):
    """
    Returns list of lists that represent indices from pivy.coin
    scene objects 'SoIndexedLine' and 'SoIndexedFace'.
    When ever a -1 is encountered in `so_node.coordIndex` a separate new Line or Face
    is created
    """
    faces = list(so_node.coordIndex)
    indices = []
    curr_line = []
    for i in faces:
        if i == -1:
            indices.append(curr_line)
            curr_line = []
            continue
        curr_line.append(i)
    return indices

def generate_line_vertices(line_indices, coord_vals):
    """
    Replaces indices in list with the corresponding coordinate values
    """
    line_vertices = []
    for i in line_indices:
        line_vertices.append(coord_vals[i])
        line_vertices.append(coord_vals[i])
    return line_vertices

def extract_values(res_tuple):
    # The names in the following comments refer to the
    # "Name" field in FreeCAD > tools > Scene Inspector
    # The types refer to the field "Inventor Tree".
    so_coord = res_tuple[1]
    so_faces = res_tuple[0] # Type: SoBrepFaceSet
    so_shaded_material = res_tuple[2]
    coords = list(so_coord.point)

    so_shaded_color = so_shaded_material.diffuseColor.getValues()[0]
    so_shaded_emissive_color = so_shaded_material.emissiveColor.getValues()[0]
    color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    emissive_color = (so_shaded_color[0], so_shaded_color[1], so_shaded_color[2])
    transparency = so_shaded_material.transparency[0]

    coord_vals = [list(x) for x in coords]
    indices = transform_indices(so_faces)
    
    is_line = False
    if type(so_faces) is coin.SoIndexedLineSet:
        is_line = True
    else:
        if not (type(so_faces) is coin.SoIndexedFaceSet):
            raise Exception("Unsupported type of given node: {}".format(type(so_faces)))
    
    so_transform = res_tuple[4]
    translation = list(so_transform.translation.getValue())
    quaternions = list(so_transform.rotation.getValue().getValue())

    #print(face_indices)
    #print(coord_vals)
    return coord_vals, indices, quaternions, translation, color, transparency, is_line

def compute_normals(faces, vertices):
    """
    Returns a list of normals for
    each vertex.
    
    Input for N faces
    should be numpy array of shape (N, 3)
    and for M vertices shape (M, 3) respectively
    """
    normals = np.zeros((len(vertices), 3), dtype='float32')
    for face in faces:
        v_index_a = face[0]
        v_index_b = face[1]
        v_index_c = face[2]
        vec_a = vertices[v_index_a]
        vec_b = vertices[v_index_b]
        vec_c = vertices[v_index_c]
        vec_a_b = np.subtract(vec_b, vec_a)
        vec_a_c = np.subtract(vec_c, vec_a)
        dot_p = np.cross(vec_a_b, vec_a_c)
        for i in [v_index_a, v_index_b, v_index_c]:
            np.add(normals[i], dot_p, normals[i])
    return normals

def create_geometry(res_tuple, name="", show_faces=True, show_lines=True):
    coord_vals, indices, quaternion, translation, color, transparency, is_line = extract_values(res_tuple)
    if is_line and show_lines:
        # geometry based on coin.IndexedLineSet
        geoms = create_line_geom(coord_vals, indices, color, translation, quaternion)
    elif not is_line and show_faces:
        # geometry based on coin.IndexedFaceSet
        geoms = create_face_geom(coord_vals, indices, color, transparency, translation, quaternion)
    else:
        return []
    if name:
        for obj in geoms:
            obj.freecad_name = name
    return geoms

def create_face_geom(coord_vals, face_indices, face_color, transparency, translation=None, quaternion=None):
    """
    Returns a pythreejs `Mesh` object that consists of the faces given by
    face_indices and the coord_vals.
    
    Additionally the attributes `Mesh.default_material` and `Mesh.geometry.default_color`
    will be set before returning the Mesh. Those attributes contain references to the
    default colors and materials that are set in this function to restore later changes.
    """
    vertices = np.asarray(coord_vals, dtype='float32')
    faces = np.asarray(face_indices, dtype='uint16')

    normals = compute_normals(faces, vertices)
        
    faces = faces.ravel()
    vertexcolors = np.asarray([face_color]*len(coord_vals), dtype='float32')


    faceGeometry = BufferGeometry(attributes=dict(
        position=BufferAttribute(vertices, normalized=False),
        index=BufferAttribute(faces, normalized=False),
        normal=BufferAttribute(normals, normalized=False),
        color=BufferAttribute(vertexcolors, normalized=False)
    ))
    # this is used for returning to original state after highlighting
    faceGeometry.default_color = vertexcolors 
    
    # BUG: This is a bug in pythreejs and currently does not work
    #faceGeometry.exec_three_obj_method('computeFaceNormals')
    #faceGeometry.exec_three_obj_method('computeVertexNormals')
    col = so_col_to_hex(face_color)
    material = MeshPhongMaterial(color=col, transparency=transparency,depthTest=True, depthWrite=True, metalness=0)
    object_mesh = Mesh(
        geometry=faceGeometry,
        material=material,
        position=[0,0,0]   # Center the cube
    )
    object_mesh.default_material = material
    
    if quaternion:
        object_mesh.quaternion = quaternion
    if translation:
        object_mesh.position = translation
    return [object_mesh]

def create_line_geom(coord_vals, indices, line_color, translation=None, quaternion=None):
    """
    Return a pythreejs Line object consisting of lines
    defined by the line_indices and the coord_vals.
    
    Additionally the attributes `Line.default_color` and `Line.edge_index`
    will be set before returning the `Line` array. Those attributes contain references to the
    default colors, that are set in this function to restore later changes, and the FreeCAD edge index.
    """
    lines = []
    for i, line_indices in enumerate(indices):
        vertices = generate_line_vertices(line_indices, coord_vals)
        vertices = np.array(vertices, dtype="float32")
        line_geom = BufferGeometry(attributes=dict(
        position=BufferAttribute(vertices, normalized=False)))
        # BUG: This is a bug in pythreejs and currently does not work
        #linesgeom.exec_three_obj_method('computeVertexNormals')
        col = so_col_to_hex(line_color)
        material = LineBasicMaterial(linewidth=LINE_WIDTH, color=col)
        material.default_color = col        
        line = Line(geometry=line_geom, 
                     material=material)
        line.edge_index = i + 1 # using FreeCAD's 1-based indexing
        if translation:
            line.position = translation
        if quaternion:
            line.quaternion = quaternion
        lines.append(line)
    return lines

def bfs_traversal(node, coordinates=None, material=None, transform=None, index=0, print_tree=False, depth_counter=0, object_index=0):
    """
    Return list of all (SoIndexed(Line/Face)Set, SoCoordinate3, SoMaterial) tuples
    inside the scene graph.
    
    The breadth first search always referes to the parent material and coordinates
    if there aren't any on the same level.
    """
    if print_tree:
        print(str("   " * index) + str(type(node)))
    if not (type(node) is coin.SoSwitch or type(node) is coin.SoSeparator):
        return []
    coords = coordinates
    mat = material
    trans = transform
    edge_face_set = None
    for child in node:
        if type(child) is coin.SoCoordinate3:
            coords = child
        if type(child) is coin.SoTransform:
            trans = child
        if type(child) is coin.SoMaterial:
            mat = child
        if type(child) is coin.SoIndexedLineSet or type(child) is coin.SoIndexedFaceSet:
            edge_face_set = child
    res_children = []
    this_object_index = -1
    for child in node:
        if depth_counter != 0:
            this_object_index = object_index
        else:
            this_object_index += 1
        res_children.extend(bfs_traversal(child, coords, transform=trans, index=index+1, print_tree=print_tree, depth_counter=depth_counter+1, object_index=this_object_index))
    if edge_face_set:
        res = [(edge_face_set, coords, mat, this_object_index, trans)]
    else:
        res = []
    res.extend(res_children)
    return res

def get_line_geometries(geometries):
    new_geometries = Group()
    for geom in list(geometries.children):
        line_geom = EdgesGeometry(geom.geometry)
        lines = LineSegments(geometry=line_geom, 
                 material=LineBasicMaterial(linewidth=5, color='#000000'))
        new_geometries.add(lines)
    return new_geometries

def get_name(obj):
    """
    Returns `object.name` except if `object is None`.
    Then returns string `"None"`.
    """
    if obj is None:
        return "None"
    return obj.name

def remove_obj_by_name(obj, children):
    """
    Removes `obj` from list `children` based on matching `obj.name`.
    If not found returns `None`.
    """
    children_new = Group()
    for obj3d in children:
        if obj3d.name == obj.name:
            continue
        children_new.add(obj3d)
    return children_new

def part_index_by_name(name, part_indices):
    """
    Returns `coin.IndexedFaceSet`s `partIndex` attribute given the
    pythreejs Geometry `name` attribute containing the objects index in the 
    Coin scene graph at the beginning of the name.
    """
    if name == "None":
        return name
    else:
        part_index = name.split()[0]
        part_index = int(part_index)
        return part_indices[part_index]
    
def index_by_face_index(part_index, face_index):
    """
    Returns the index of the Shape face for a given face index.
    If the face index is not in the part_index returns `None`.
    """
    upper_limit = 0
    for i, part_num_elements in enumerate(part_index):
        upper_limit += part_num_elements
        if face_index < upper_limit:
            return i + 1 # FreeCAD uses 1-based indexing
    return None

def vertices_col_highlight_face(shape_face_index, cols_default, part_index, face_indices):
    """
    Returns vertex color array where the indexed face is highlighted.
    """
    cols = np.copy(cols_default)
    start_index = 0
    for i in range(shape_face_index-1): #this is using the freecad base 1-indexing!
        start_index += part_index[i]*3
    end_index = start_index + part_index[shape_face_index-1]*3 - 1
    if start_index < 0:
        start_index = 0
    for i in range(start_index, end_index+1):
        pos_index = face_indices[i]
        cols[pos_index] = HIGHLIGHTING_COLOR
    return cols

def reset_object_highlighting(obj):
    """
    After calling this function the object of type `Line`, `Sphere` or `Mesh` will be reset
    to it's default colors.
    """
    # case obj is a Line
    if type(obj) is Line:
        obj.material.color = obj.material.default_color
        return
    
    # case obj is a Sphere (representing a vertex)
    if type(obj) is Sphere:
        return
    
    # case obj is a Mesh of faces
    obj.material = obj.default_material
    obj.geometry.attributes["color"].array = obj.geometry.default_color
    obj.geometry.attributes["color"].needsUpdate = True        

    
def generate_picker(geometries, scene, part_indices, mode="click"):
    """
    Returns a picker that will enable object and face selection
    as well as highlighting those selections
    
    Picker mode can be `mousemove` or `click` or `dblclick` and is set via the `mode`
    parameter.
    """
    VALUE_TYPE = "point"
    VALID_MODES = ["mousemove", "click", "dblclick"]
    if mode not in VALID_MODES:
        raise Exception("Given `mode` parameter has to be on of {}, but was `{}`"
                        .format(VALID_MODES, mode))
    html = HTML("<b>No selection.</b>")
    picker = Picker(
        controlling = geometries,
        event = mode)
    picker.shape_face_index_old = -1
    picker.last_object = None
    
    def f(change):
        """
        This functions implements highlighting and displaying the name 
        of selected faces and edges as well as vertices. You seemingly can't
        select nothing, it only triggers on objects.
        """
        value = picker.object
        last_value = picker.last_object
        
        txt = ""
        if hasattr(value, "freecad_name"):
            txt = "{}: ".format(value.freecad_name)
        html.value = "{}".format(txt)
        
        if type(value) is Line:
            if not (last_value is None):
                reset_object_highlighting(last_value)
            value.material.color = so_col_to_hex(HIGHLIGHTING_COLOR)
            html.value += "<b>Edge{}</b>".format(value.edge_index)#value.edge_index)
            return

        if value is None:
            if not (last_value is None):
                reset_object_highlighting(last_value)
            html.value += "<b>No selection.</b>"
            return

        face_index = int(picker.faceIndex)
        geom = value.geometry
        part_index = part_index_by_name(get_name(value), part_indices)
        shape_face_index = index_by_face_index(part_index, face_index)        

        if not (last_value is None):
            # check for case of selecting the same freecad face
            if (last_value.name == value.name) and (picker.shape_face_index_old == shape_face_index):
                return
            reset_object_highlighting(last_value)

        face_indices = value.geometry.attributes["index"].array
        cols_default = value.geometry.default_color
        cols_highlighted = vertices_col_highlight_face(shape_face_index, cols_default, part_index, face_indices)
        value.geometry.attributes["color"].array = cols_highlighted
        value.geometry.attributes["color"].needsUpdate = True 
        
        material = MeshLambertMaterial(vertexColors='VertexColors', shininess=1)
        material.name = value.material.color
        value.material = material
        picker.shape_face_index_old = shape_face_index
        picker.last_object = value

        html.value += "<b>Face{}</b>".format(shape_face_index)

    picker.observe(f, names=[VALUE_TYPE]) 
    return html, picker

def render_objects(root_node, names=[], show_line_geom=False, show_normals=False):
    """
    Renders any coin node containing LineSets or FaceSets.
    
    show_line_geom: Just display the Mesh.
    show_normals: Display the vertex normals.
    """
    view_width = 600
    view_height = 600
    geometries = Group()
    part_indices = [] # contains the partIndex indices that relate triangle faces to shape faces
    render_face_set = True
    i = 0
    for res in bfs_traversal(root_node, print_tree=False):
        if type(res[0]) is coin.SoIndexedFaceSet and render_face_set:
            render_face_set = False
            continue
        elif type(res[0]) is coin.SoIndexedFaceSet:
            render_face_set = True
            part_index_list = list(res[0].partIndex)
            part_indices.append(part_index_list)
        elif type(res[0]) is coin.SoIndexedLineSet:
            pass
        else:
            continue
        if False:
            geoms = create_geometry(res)
        else:
            geoms = create_geometry(res)
        for obj3d in geoms:
            obj3d.name = str(res[3]) + " " + str(i) #the name of the object is `object_index i`
            i += 1
        if geoms and show_normals:
            helper = VertexNormalsHelper(geom[0])
            geoms.append(helper)
        for geom in geoms:
            geometries.add(geom)
    
    if show_line_geom:
        geometries = get_line_geometries(geometries)
        
    light = PointLight(color="white", position=[40,40,40], intensity=1.0, castShadow=True)
    fog = Fog(color="#3f7b9d")
    ambient_light = AmbientLight(intensity=0.5)
    camera = PerspectiveCamera(
        position=[0, -40, 20], fov=40,
        aspect=view_width/view_height)
    children = [camera, light, ambient_light]
    children.append(geometries)
    scene = Scene(children=children)
    scene.background = "#65659a"
  
    html, picker = generate_picker(geometries, scene, part_indices, "mousemove") 
    controls = [OrbitControls(controlling=camera), picker]   
    renderer = Renderer(camera=camera,
                    scene=scene, controls=controls,
                    width=view_width, height=view_height)
    return display(renderer, html)


def document_to_scene_graph(doc):
    root = coin.SoSeparator()
    names = []
    for obj in doc.Objects:
        root.addChild(FreeCADGui.subgraphFromObject(obj))
        names.append(obj.Name)
    return root, names

def render_document(doc):
    root, names = document_to_scene_graph(doc)
    return render_objects(root, names)

## Importing FreeCAD

Now we can verify that these functions do what they are supposed to do. First we add the JUPYTER_REPO_PATH (path to the Github repository this file is part of. Not necessary right now, but this will be used at a later stage of development) and the FreeCAD shared library path. Then we just import FreeCAD and set it up for headless usage. No firing up the desktop app!

In [3]:
import sys, os

JUPYTER_REPO_PATH = "/opt/jupyter_freecad/"

sys.path.append("/opt/freecad/freecad_build/lib")
sys.path.append(JUPYTER_REPO_PATH + "Jupyter")

import FreeCAD, FreeCADGui
FreeCADGui.setupWithoutGUI()

Creating a document with objects and a scene graph to be iterated over later on.

In [4]:
doc = FreeCAD.newDocument("test_document")
doc.addObject("Part::Box","Box")
doc.addObject("Part::Cylinder","Cylinder")
doc.addObject("Part::Sphere","Sphere")
doc.addObject("Part::Torus","Torus")
doc.recompute()
doc.FileName = "test_document.FCStd"
doc.save()

Now if everything works as expected this is all we need to render the 3D view right in the notebook:

TO DO:

the `part_indices` value is correct. Now We need to match the `faceIndex` to the `partIndex` and then hightlight all the faces beloging to the `partIndex`.

In [5]:
render_document(doc)

Renderer(camera=PerspectiveCamera(fov=40.0, position=(0.0, -40.0, 20.0), quaternion=(0.0, 0.0, 0.0, 1.0), scal…

HTML(value='<b>No selection.</b>')

# Taking advantage of Ipython plugins
## ipywidgets example

The following code will display an interactive view that will modify FreeCAD document content based on interacive user input via the ipywidget UI elements.

In [6]:
from ipywidgets import interact
import ipywidgets as widgets
doc = FreeCAD.newDocument()
doc.addObject("Part::Box","Box")
def create_scene(box, cylinder, sphere, torus):
    if box:
        if not doc.getObject("Box"):
            doc.addObject("Part::Box","Box")
    elif doc.getObject("Box"):
        doc.removeObject("Box")
    if cylinder:
        if not doc.getObject("Cylinder"):
            doc.addObject("Part::Cylinder","Cylinder")
    elif doc.getObject("Cylinder"):
        doc.removeObject("Cylinder")
    if sphere:
        if not doc.getObject("Sphere"):
            doc.addObject("Part::Sphere","Sphere")
    elif doc.getObject("Sphere"):
        doc.removeObject("Sphere")
    if torus:
        if not doc.getObject("Torus"):
            doc.addObject("Part::Torus","Torus")
    elif doc.getObject("Torus"):
        doc.removeObject("Torus")
    doc.recompute()

    root = coin.SoSeparator()
    for obj in doc.Objects:
        root.addChild(FreeCADGui.subgraphFromObject(obj))
    return render_objects(root)

interact(create_scene, box=True, cylinder=False, sphere=False, torus=False);

interactive(children=(Checkbox(value=True, description='box'), Checkbox(value=False, description='cylinder'), …

# Opening the previously saved document

In [7]:
doc = FreeCAD.openDocument(JUPYTER_REPO_PATH + "test_document.FCStd")
render_document(doc)

Renderer(camera=PerspectiveCamera(fov=40.0, position=(0.0, -40.0, 20.0), quaternion=(0.0, 0.0, 0.0, 1.0), scal…

HTML(value='<b>No selection.</b>')

# Opening a file created with the FreeCAD dektop app

Created with FreeCAD 0.18 on MacOS

In [8]:
doc = FreeCAD.openDocument(JUPYTER_REPO_PATH + "test_freecad_0.18_macos.FCStd")
render_document(doc)

Renderer(camera=PerspectiveCamera(fov=40.0, position=(0.0, -40.0, 20.0), quaternion=(0.0, 0.0, 0.0, 1.0), scal…

HTML(value='<b>No selection.</b>')

In [9]:
root, names = document_to_scene_graph(doc)
list(root[2][0].translation.getValue())
#list(root[0][0].rotation.getValue().getValue())

[1.8148322105407715, 6.1183953285217285, -0.013501799665391445]

# Test cases

 - Fix edges
 - Regression tests for workbenches (to test compatibility)
 - First selecting entire objects, highlight selection
 - Have a constant place in the UI that shows a copy pastable label text

# Experiments for debugging

In [10]:
help(FreeCAD.ActiveDocument)

Help on Document object:

class Document(PropertyContainer)
 |  This is a Document class
 |  
 |  Method resolution order:
 |      Document
 |      PropertyContainer
 |      Base.Persistence
 |      Base.BaseClass
 |      builtins.PyObjectBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |  
 |  abortTransaction(...)
 |      Abort an Undo/Redo transaction (rollback)
 |  
 |  addObject(...)
 |      addObject(type, name=None, objProxy=None, viewProxy=None, attach=False, viewType=None)
 |      
 |      Add an object to document
 |      
 |      type (String): the type of the 

In [11]:
vertices = np.array([(0,0,0), (10,10,10)], dtype="float32")
vertexcolors = np.asarray([(1,0,0)]*len(vertices), dtype='float32')
line_geom = BufferGeometry(attributes=dict(
            position=BufferAttribute(vertices, normalized=False)))
col = so_col_to_hex((1,0,0))
material = LineBasicMaterial(linewidth=LINE_WIDTH, color="#ff0000")
material.default_color = col        
line = Line(geometry=line_geom, material=material)
print(line)

Line(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=array([[ 0.,  0.,  0.],
       [10., 10., 10.]], dtype=float32))}), material=LineBasicMaterial(color='#ff0000', linewidth=10.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))


In [12]:
geometries = Group()
light = PointLight(color="white", position=[40,40,40], intensity=1.0, castShadow=True)
fog = Fog(color="#3f7b9d")
ambient_light = AmbientLight(intensity=0.5)
camera = PerspectiveCamera(
        position=[0, -40, 20], fov=40,
        aspect=1)
children = [camera, light, ambient_light]
geometries.add(line)
geometries.add(line)
children.append(geometries)
scene = Scene(children=children)
scene.background = "#65659a"
  
controls = [OrbitControls(controlling=camera)]   
renderer = Renderer(camera=camera,
                    scene=scene, controls=controls,
                    width=600, height=600)
display(renderer)

Renderer(camera=PerspectiveCamera(fov=40.0, position=(0.0, -40.0, 20.0), quaternion=(0.0, 0.0, 0.0, 1.0), scal…

In [13]:
line.material.color = "#00ff00"